In [ ]:
import math as m

In [ ]:
def k_mers(s,k):
  kmers=[]
  for i in range(len(s)-k+1):
    kmers.append(s[i:i+k])
  return kmers

k_mers("AAACGCT",2)

In [ ]:
# Funzione che conta il numero di occorrenze di una parola in una sequenza
def count(w,s):
  c=0
  for i in range(len(s)-len(w)+1):
    if w==s[i:i+len(w)]:
      c+=1
  return c

In [ ]:
# Funziona che crea una lista contenente tutte le parole di lunghezza k da un certo vocabolario
def All(s,k):
  if k==1:
    return s
  output=[]
  for i in s:
    for j in All(s,k-1):
      output.append(i+j)
  return output

All("ACGT",3)

In [ ]:
# Funzione che crea il vettore delle occorrenze 
def h(s,k):
  output={}
  for w in All("ACGT",k):
    output[w]=0
  for w in k_mers(s,k):
    output[w]=count(w,s)
  return output

s="AACCAA"
k=2
x=h(s,k)

In [ ]:
for p in x.values():
  print(p)

In [ ]:
# Funzione che implementa la distanza euclidea tra due stringhe
def euclidean(s,t,k):
  if len(s)==len(t):
   x=h(s,k)
   x1=[]
   for p in x.values():
     x1.append(p)
   y=h(t,k)
   y1=[]
   for p in y.values():
     y1.append(p)
   d=0
   for i in range(len(s)):
     d+=(x1[i]-y1[i])**2
  return d



In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [ ]:
findspark.init()

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar xvfz ngrok-stable-linux-amd64.tgz

In [ ]:
!pip install pyngrok

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName("GEBD 2122")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
        
sc = spark.sparkContext

In [ ]:
!killall ngrok

In [ ]:
!./ngrok authtoken 22MppDcRusuK7c4t3UoVIgZ0W00_5WGzspSFKUHYNyETBXZGE 

In [ ]:
!./ngrok http 4050

In [ ]:
from pyngrok import ngrok

#publ_url = ngrok.connect(addr='4050')
#ngrok.disconnect(publ_url.public_url)
publ_url = ngrok.connect(4050,'http')
print(publ_url)

In [ ]:
b1 = sc.textFile("NC_009057.fasta")
b2 = sc.textFile("NC_009063.fasta")


kmer1 = b1.filter(lambda x: '>NC_009057' not in x)
kmer1=kmer1.map(lambda x: x.lower().strip(',.;:!?()-')).flatMap(lambda x: k_mers(x,15)).map(lambda x: (x,1))
kmer1=kmer1.reduceByKey(lambda x,y: x+y)

kmer2 = b2.filter(lambda x: '>NC_009063' not in x)
kmer2=kmer2.map(lambda x: x.lower().strip(',.;:!?()-')).flatMap(lambda x: k_mers(x,15)).map(lambda x: (x,1))
kmer2=kmer2.reduceByKey(lambda x,y: x+y)


# Distanza euclidea
kmer=kmer1.union(kmer2).reduceByKey(lambda x,y: (x-y)**2)

print(m.sqrt(kmer.values().sum()))

# Distanza D2
kmer=kmer1.union(kmer2).reduceByKey(lambda x,y: x*y)

print(kmer.values().sum())